In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
n_houses = 1000
df=pd.DataFrame(columns=['id','closing_price', 'agent_1', 'agent_2', 'agent_3'])

In [3]:
n_houses = 1000
df=pd.DataFrame()
df["id"] = range(n_houses)
df[['closing_price', 'agent_1', 'agent_2', 'agent_3']]= np.random.uniform(80000,2000000,(n_houses,4))
df


,id,closing_price,agent_1,agent_2,agent_3
0,0,9.913723e+05,1.527132e+06,1.561870e+06,1.679284e+05
1,1,1.997428e+06,4.288292e+05,1.839622e+06,1.479262e+05
2,2,1.137832e+06,8.527096e+05,5.399991e+05,1.920775e+06
3,3,1.192087e+06,7.059543e+05,7.786834e+05,1.994901e+06
4,4,2.053307e+05,2.658393e+05,2.758467e+05,9.841670e+05
...,...,...,...,...,...
995,995,1.380208e+06,1.978761e+06,1.277041e+06,1.708322e+06
996,996,8.144941e+05,1.951244e+06,6.434645e+05,1.602027e+06
997,997,1.632120e+06,1.240640e+06,6.689857e+05,2.372524e+05
998,998,1.264569e+06,5.083779e+05,2.298767e+05,1.295094e+06


In [4]:
df_long = df.melt(id_vars=["id"], value_vars=['closing_price','agent_1', 'agent_2', 'agent_3'], var_name="agent", value_name="valuation")
df_long
df_wide = df_long.pivot(index="id", columns="agent",values="valuation")
df_wide.columns.name=None
df_wide.reset_index(inplace=True)
df_wide

,id,agent_1,agent_2,agent_3,closing_price
0,0,1.527132e+06,1.561870e+06,1.679284e+05,9.913723e+05
1,1,4.288292e+05,1.839622e+06,1.479262e+05,1.997428e+06
2,2,8.527096e+05,5.399991e+05,1.920775e+06,1.137832e+06
3,3,7.059543e+05,7.786834e+05,1.994901e+06,1.192087e+06
4,4,2.658393e+05,2.758467e+05,9.841670e+05,2.053307e+05
...,...,...,...,...,...
995,995,1.978761e+06,1.277041e+06,1.708322e+06,1.380208e+06
996,996,1.951244e+06,6.434645e+05,1.602027e+06,8.144941e+05
997,997,1.240640e+06,6.689857e+05,2.372524e+05,1.632120e+06
998,998,5.083779e+05,2.298767e+05,1.295094e+06,1.264569e+06


In [5]:
df_calculations = df_long.merge(right=df[["id","closing_price"]], how="left", on="id")
df_calculations = df_calculations[df_calculations['agent']!='closing_price']
df_calculations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 1000 to 3999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             3000 non-null   int64  
 1   agent          3000 non-null   object 
 2   valuation      3000 non-null   float64
 3   closing_price  3000 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 117.2+ KB


In [6]:
df_calculations.reset_index(inplace=True,drop=True)
df_calculations['se'] = (df_calculations['closing_price']-df_calculations['valuation'])**2
rmse=df_calculations.groupby(by=['agent']).agg({"se":[np.mean]})**0.5
rmse.rename(columns={"se":"rmse"}, level=0, inplace=True)
rmse.droplevel(level=1,axis=1).reset_index()

,agent,rmse
0,agent_1,770992.359272
1,agent_2,761946.284385
2,agent_3,791080.702413


In [7]:
df_calculations.sample(frac=0.8,axis=0, replace=False)

,id,agent,valuation,closing_price,se
733,733,agent_1,1.515630e+06,1.577846e+06,3.870852e+09
1169,169,agent_2,1.168701e+06,1.227834e+06,3.496715e+09
1764,764,agent_2,9.476392e+05,1.088463e+06,1.983144e+10
2925,925,agent_3,1.177496e+06,1.115106e+06,3.892454e+09
2946,946,agent_3,1.540086e+06,1.558491e+05,1.916111e+12
...,...,...,...,...,...
800,800,agent_1,1.701244e+06,8.592450e+05,7.089628e+11
2037,37,agent_3,1.503174e+06,1.225842e+06,7.691318e+10
2380,380,agent_3,9.925072e+05,5.761838e+05,1.733252e+11
1051,51,agent_2,1.367531e+06,1.894708e+06,2.779146e+11


In [8]:
rng = np.random.default_rng(seed=123)

mask = rng.choice(df_calculations.index, size=int(round(0.8*df_calculations.shape[0],0)), replace=False)
df_calculations.loc[mask].shape

(2400, 5)

In [9]:
df['closing_price']

0      9.913723e+05
1      1.997428e+06
2      1.137832e+06
3      1.192087e+06
4      2.053307e+05
           ...     
995    1.380208e+06
996    8.144941e+05
997    1.632120e+06
998    1.264569e+06
999    1.021466e+05
Name: closing_price, Length: 1000, dtype: float64

In [10]:
df.apply(lambda x: x['agent_1']-x['closing_price'] , axis=1)

0      5.357597e+05
1     -1.568598e+06
2     -2.851219e+05
3     -4.861327e+05
4      6.050861e+04
           ...     
995    5.985529e+05
996    1.136750e+06
997   -3.914797e+05
998   -7.561913e+05
999    1.832786e+05
Length: 1000, dtype: float64

In [11]:
df.transform(func=lambda x: x-x.mean(), axis=0)

,id,closing_price,agent_1,agent_2,agent_3
0,-499.5,-46877.375708,507707.054615,556798.255416,-876770.915620
1,-498.5,959177.966717,-590595.759990,834550.266739,-896773.126562
2,-497.5,99581.872713,-166715.301196,-465072.591479,876076.051914
3,-496.5,153837.407966,-313470.596343,-226388.263745,950201.588063
4,-495.5,-832918.940678,-753585.620614,-729225.017806,-60532.233009
...,...,...,...,...,...
995,495.5,341958.581265,959336.243892,271968.884358,663622.634682
996,496.5,-223755.533282,931819.561248,-361607.245551,557327.523424
997,497.5,593870.017217,221215.062575,-336086.031646,-807446.879634
998,498.5,226319.629067,-511046.976962,-775195.052353,250394.234439


In [12]:
data = pd.DataFrame({'EmployeeName': ['Callen Dunkley', 'Sarah Rayner', 'Jeanette Sloan', 'Kaycee Acosta', 'Henri Conroy', 'Emma Peralta', 'Martin Butt', 'Alex Jensen', 'Kim Howarth', 'Jane Burnett'],
                    'Department': ['Accounting', 'Engineering', 'Engineering', 'HR', 'HR', 'HR', 'Data Science', 'Data Science', 'Accounting', 'Data Science'],
                    'HireDate': [2010, 2018, 2012, 2014, 2014, 2018, 2020, 2018, 2020, 2012],
                    'Sex': ['M', 'F', 'F', 'F', 'M', 'F', 'M', 'M', 'M', 'F'],
                    'Birthdate': ['04/09/1982', '14/04/1981', '06/05/1997', '08/01/1986', '10/10/1988', '12/11/1992', '10/04/1991', '16/07/1995', '08/10/1992', '11/10/1979'],
                    'Weight': [78, 80, 66, 67, 90, 57, 115, 87, 95, 57],
                    'Height': [176, 160, 169, 157, 185, 164, 195, 180, 174, 165],
                    'Kids': [2, 1, 0, 1, 1, 0, 2, 0, 3, 1],
                     'mobile' : ['+1 224 388 1527','224-388-1527','1 2243881527','12243881527','224388 1527','+2243881527','+2243881527',
                                 '$1-224 388 1527','1334 388 1527','224$388 1527']
                    })
data.to_csv('./df_employees.csv')

In [13]:
def age_cal(Birthdate):
  
  today = pd.to_datetime('today')
  age = today.year - Birthdate.year - ( (today.month, today.day) < (Birthdate.month, Birthdate.day) )
  return age

In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.getOrCreate()
dir(spark.read)    # to see an object's methods
print(spark.read.__doc__)


    Interface used to load a :class:`DataFrame` from external storage systems
    (e.g. file systems, key-value stores, etc). Use :attr:`SparkSession.read`
    to access this.

    .. versionadded:: 1.4
    


In [33]:

employees = spark.read.csv('./df_employees.csv', header=True)
employees.printSchema()
employees.show(n=5)

root
 |-- _c0: string (nullable = true)
 |-- EmployeeName: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- HireDate: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Kids: string (nullable = true)
 |-- mobile: string (nullable = true)

+---+--------------+-----------+--------+---+----------+------+------+----+---------------+
|_c0|  EmployeeName| Department|HireDate|Sex| Birthdate|Weight|Height|Kids|         mobile|
+---+--------------+-----------+--------+---+----------+------+------+----+---------------+
|  0|Callen Dunkley| Accounting|    2010|  M|04/09/1982|    78|   176|   2|+1 224 388 1527|
|  1|  Sarah Rayner|Engineering|    2018|  F|14/04/1981|    80|   160|   1|   224-388-1527|
|  2|Jeanette Sloan|Engineering|    2012|  F|06/05/1997|    66|   169|   0|   1 2243881527|
|  3| Kaycee Acosta|         HR|    2014|  F|08/01/1986

In [16]:
results = (
spark.read.csv("./df_employees.csv", header=True)
.select(F.split(F.col("EmployeeName"), " ").alias("line"))
.select(F.explode(F.col("line")).alias("word"))
.select(F.lower(F.col("word")).alias("word"))
.select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
.where(F.col("word") != "")
.groupby("word")
.count()
)
#results.show(n=20)
#results.write.csv('./results_single_partition.csv')
results.coalesce(1).write.csv('./data/results_single_partition.csv')  # Writing our results under a single partition

'''
# Creating our own simple SparkSession for batch submit
from pyspark.sql import SparkSession
spark = (SparkSession.builder
.appName("Counting word occurences from a book.")
.getOrCreate())

spark-submit ./code/Ch02/word_count_submit.py
'''

Py4JJavaError: An error occurred while calling o69.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\Users\oscar\spark\bin\bin -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:287)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at

In [27]:
my_grocery_list = [
["Banana", 2, 1.74],
["Apple", 4, 2.04],
["Carrot", 1, 1.09],
["Cake", 1, 10.99],
]
df_grocery_list = spark.createDataFrame(my_grocery_list, ["Item", "Quantity", "Price"])
df_grocery_list.printSchema()

In [36]:
employees.select("EmployeeName").show(5, False)
employees2 = employees.drop("EmployeeName")
employees2.show()

+--------------+
|EmployeeName  |
+--------------+
|Callen Dunkley|
|Sarah Rayner  |
|Jeanette Sloan|
|Kaycee Acosta |
|Henri Conroy  |
+--------------+
only showing top 5 rows

+---+------------+--------+---+----------+------+------+----+---------------+
|_c0|  Department|HireDate|Sex| Birthdate|Weight|Height|Kids|         mobile|
+---+------------+--------+---+----------+------+------+----+---------------+
|  0|  Accounting|    2010|  M|04/09/1982|    78|   176|   2|+1 224 388 1527|
|  1| Engineering|    2018|  F|14/04/1981|    80|   160|   1|   224-388-1527|
|  2| Engineering|    2012|  F|06/05/1997|    66|   169|   0|   1 2243881527|
|  3|          HR|    2014|  F|08/01/1986|    67|   157|   1|    12243881527|
|  4|          HR|    2014|  M|10/10/1988|    90|   185|   1|    224388 1527|
|  5|          HR|    2018|  F|12/11/1992|    57|   164|   0|    +2243881527|
|  6|Data Science|    2020|  M|10/04/1991|   115|   195|   2|    +2243881527|
|  7|Data Science|    2018|  M|16/07/1995|

In [40]:
employees.select(
    "Birthdate",
    F.col("Birthdate").substr(1, 2).cast("int").alias("day"),
    F.col("Birthdate").substr(4, 2).cast("int").alias("month"),
    F.col("Birthdate").substr(7, 4).cast("int").alias("year"),
    ).distinct().show()

employees.withColumn("day",F.col("Birthdate").substr(1, 2).cast("int")).show() # another way to create a column

+---+--------------+------------+--------+---+----------+------+------+----+---------------+---+
|_c0|  EmployeeName|  Department|HireDate|Sex| Birthdate|Weight|Height|Kids|         mobile|day|
+---+--------------+------------+--------+---+----------+------+------+----+---------------+---+
|  0|Callen Dunkley|  Accounting|    2010|  M|04/09/1982|    78|   176|   2|+1 224 388 1527|  4|
|  1|  Sarah Rayner| Engineering|    2018|  F|14/04/1981|    80|   160|   1|   224-388-1527| 14|
|  2|Jeanette Sloan| Engineering|    2012|  F|06/05/1997|    66|   169|   0|   1 2243881527|  6|
|  3| Kaycee Acosta|          HR|    2014|  F|08/01/1986|    67|   157|   1|    12243881527|  8|
|  4|  Henri Conroy|          HR|    2014|  M|10/10/1988|    90|   185|   1|    224388 1527| 10|
|  5|  Emma Peralta|          HR|    2018|  F|12/11/1992|    57|   164|   0|    +2243881527| 12|
|  6|   Martin Butt|Data Science|    2020|  M|10/04/1991|   115|   195|   2|    +2243881527| 10|
|  7|   Alex Jensen|Data Scien

In [59]:
# rename columns 
#employees.withColumnRenamed("Birthdate", "birthdate").printSchema()
#employees.toDF(*[x.lower() for x in employees.columns]).printSchema()

# select in alphabetical order
#employees.select(sorted(employees.columns)).printSchema()

## describe
#employees.describe().show()
#employees.describe("EmployeeName").show()
#for i in employees.columns:
#    employees.describe(i).show()

## summary
#employees.summary().show()
#employees.select("EmployeeName").summary().show()
#employees.select("EmployeeName").summary("min", "10%", "90%", "max").show()

## joins
#[LEFT].join(
#[RIGHT],
#on=[PREDICATES]
#how=[METHOD]  )   left, right, inner, outer,
#)

## agg
#employees.groupBy("Department").count().show
#employees.groupBy("Department").agg(F.sum("Kids").alias("total_kids")).show()

## other
#employees.dropna(subset=["Department"]).show()
#employees.fillna(0).show()  # can use a dictionary as in python




+---+--------------+------------+--------+---+----------+------+------+----+---------------+
|_c0|  EmployeeName|  Department|HireDate|Sex| Birthdate|Weight|Height|Kids|         mobile|
+---+--------------+------------+--------+---+----------+------+------+----+---------------+
|  0|Callen Dunkley|  Accounting|    2010|  M|04/09/1982|    78|   176|   2|+1 224 388 1527|
|  1|  Sarah Rayner| Engineering|    2018|  F|14/04/1981|    80|   160|   1|   224-388-1527|
|  2|Jeanette Sloan| Engineering|    2012|  F|06/05/1997|    66|   169|   0|   1 2243881527|
|  3| Kaycee Acosta|          HR|    2014|  F|08/01/1986|    67|   157|   1|    12243881527|
|  4|  Henri Conroy|          HR|    2014|  M|10/10/1988|    90|   185|   1|    224388 1527|
|  5|  Emma Peralta|          HR|    2018|  F|12/11/1992|    57|   164|   0|    +2243881527|
|  6|   Martin Butt|Data Science|    2020|  M|10/04/1991|   115|   195|   2|    +2243881527|
|  7|   Alex Jensen|Data Science|    2018|  M|16/07/1995|    87|   180